In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


df = pd.read_csv('/Users/cstone/Documents/Projects/MLB_Pitch_Prediction/data/raw/blake_snell.csv')

df.head()


,Unnamed: 0,pitch_type,balls,strikes,on_3b,on_2b,on_1b,outs_when_up,inning,at_bat_number,...,bat_score,fld_score,delta_home_win_exp,delta_run_exp,batter_is_right,pitcher_is_right,inning_top,if_fielding_alignment_Standard,if_fielding_alignment_Strategic,of_fielding_alignment_Strategic
0,0,CU,0,2,0.0,1.0,1.0,2,6,47,...,0,1,-0.077,-0.311,1,0,0,True,False,False
1,1,FF,0,1,0.0,1.0,1.0,2,6,47,...,0,1,0.000,-0.098,1,0,0,True,False,False
2,2,FF,0,0,0.0,1.0,1.0,2,6,47,...,0,1,0.000,-0.063,1,0,0,True,False,False
3,3,CH,1,0,0.0,0.0,1.0,2,6,46,...,0,1,0.035,0.284,1,0,0,True,False,False
4,4,CU,0,0,0.0,0.0,1.0,2,6,46,...,0,1,0.000,0.025,1,0,0,True,False,False


In [2]:
df.columns

Index(['Unnamed: 0', 'pitch_type', 'balls', 'strikes', 'on_3b', 'on_2b',
       'on_1b', 'outs_when_up', 'inning', 'at_bat_number', 'pitch_number',
       'home_score', 'away_score', 'bat_score', 'fld_score',
       'delta_home_win_exp', 'delta_run_exp', 'batter_is_right',
       'pitcher_is_right', 'inning_top', 'if_fielding_alignment_Standard',
       'if_fielding_alignment_Strategic', 'of_fielding_alignment_Strategic'],
      dtype='object')

In [3]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score


Train test split

In [4]:
X = df.loc[:, df.columns!='pitch_type']
y = df['pitch_type']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

Scale the data

In [5]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Test Accuracy

In [6]:
#testing different values of K to get optimal parameter
for i in range(1,20):

    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{i} Accuracy:", accuracy)

1 Accuracy: 0.4275092936802974
2 Accuracy: 0.3550185873605948
3 Accuracy: 0.3940520446096654
4 Accuracy: 0.42379182156133827
5 Accuracy: 0.43308550185873607
6 Accuracy: 0.42193308550185876
7 Accuracy: 0.44423791821561337
8 Accuracy: 0.45539033457249073
9 Accuracy: 0.4721189591078067
10 Accuracy: 0.47769516728624534
11 Accuracy: 0.483271375464684
12 Accuracy: 0.48698884758364314
13 Accuracy: 0.4944237918215613
14 Accuracy: 0.5
15 Accuracy: 0.4962825278810409
16 Accuracy: 0.5055762081784386
17 Accuracy: 0.5037174721189591
18 Accuracy: 0.5055762081784386
19 Accuracy: 0.5018587360594795


Cross Validate

In [14]:


knn_cv = KNeighborsClassifier(n_neighbors=24)
cv_scores = cross_val_score(knn_cv, X, y, cv= 5)
print(cv_scores)
print("cv_scores mean:{}".format(np.mean(cv_scores)))

[0.20817844 0.09107807 0.0744879  0.13780261 0.46927374]
cv_scores mean:0.1961641502772528


GridSearch CV to ensure best k-value is selected 

In [13]:
from sklearn.model_selection import GridSearchCV

knn2 = KNeighborsClassifier()
param_grid = {"n_neighbors": np.arange(1, 25)}
knn_gscv = GridSearchCV(knn2, param_grid, cv=5)


knn_gscv.fit(X, y)
knn_gscv.best_params_

{'n_neighbors': 24}

Overall accuracy using KNN seems fairly low at ~20%. Given a pitcher has 5 pitches, this isnt much better than just guessing. 